In [7]:
# Real + Imaginary together as one feature
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC


# Load dataset
measurement = np.load('../../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:200]
n_comp = 4

# Train-test split
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)
print(f'trainData - {trainCIR.shape}')
print(f'testData - {testCIR.shape}')

# Define channels
alice_channel = 3  # A -> B (legitimate)
eve_channel = 6  # E -> B (illegitimate)

trainData - (160, 15, 251, 2)
testData - (40, 15, 251, 2)


In [8]:
# -------------------------------------------------- Preprocessing ------------------------------------------
import matplotlib.pyplot as plt

def reduce_data_dimension(data, n_components):
    # data: (samples, 251, 2)
    
    reformed_cirs = []    
    for idx, cir in enumerate(data):
        # if idx == 1:
        #     break
        
        products = np.abs(cir[:, 0] * cir[:, 1])
        print(f'products - {products}')
        # products = [12, 5, 0, 6, 12]

        # np.argsort(products) = [2, 1, 3, 0, 4] => value => [0 , 5, 6, 12, 12]
        # np.argsort(products)[::-1] = [4, 0, 3, 1, 2] index in reverse order
        top_indices = np.argsort(products)[::-1][:n_components]
        print(f'top_indices - {top_indices}')
        top_components = cir[top_indices]
        print(f'top_components - {top_components}')
        reformed_cirs.append(top_components)
        
        # draw CIR ------------
        # # Plot the entire CIR (251 data points)
        # plt.plot(range(251), products, label='All Data Points', color='blue')
        # # Highlight the 4 most important data points
        # plt.scatter(top_indices, products[top_indices], color='red', label='Top 4 Points', s=100)
        
        # # Labeling the plot
        # plt.xlabel('Data Point Index')
        # plt.ylabel('Importance Metric (Real * Imaginary)')
        # plt.title('Comparison of All CIR Points with Top 4 Important Points')
        # plt.legend()
        # plt.show()

    reformed_cirs = np.array(reformed_cirs)
    
    return reformed_cirs

# -------- Train --------
# Feature Extraction
train_alice_cirs = trainCIR[:, alice_channel, :, :]  # (6400, 251, 2)
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # (6400, 251, 2)
train_cirs = np.vstack((train_alice_cirs, train_eve_cirs))  # (12800, 251, 2)

# Dimension Redzuction
train_cirs_reduced = reduce_data_dimension(train_cirs, n_components=n_comp)  # (12800, n_components)
# Flatten for Classifier
train_cirs_flattened = train_cirs_reduced.reshape(train_cirs_reduced.shape[0], -1)  # (12800, 8)
print(f'train_cirs_pca - {train_cirs_reduced.shape}')

# Labels
train_alice_labels = np.zeros(train_alice_cirs.shape[0])  # '0' for Alice
train_eve_labels = np.ones(train_eve_cirs.shape[0])       # '1' for Eve
train_labels = np.hstack((train_alice_labels, train_eve_labels))  # (12800,)
print(f'train_labels - {train_labels.shape}')

# -------- Test --------
# Feature Extraction
test_alice_cirs = testCIR[:, alice_channel, :, :]  # (1600, 251, 2)
test_eve_cirs = testCIR[:, eve_channel, :, :]      # (1600, 251, 2)
test_cirs = np.vstack((test_alice_cirs, test_eve_cirs))  # (3200, 251, 2)
test_cirs_reduced = reduce_data_dimension(test_cirs, n_components=n_comp)  # (12800, n_components)
# Flatten for Classifier
test_cirs_flattened = test_cirs_reduced.reshape(test_cirs_reduced.shape[0], -1)  # (3200, 8)
print(f'test_cirs_pca - {test_cirs_reduced.shape}')

# Labels
test_alice_label = np.zeros(test_alice_cirs.shape[0])  # '0' for Alice
test_eve_labels = np.ones(test_eve_cirs.shape[0])       # '1' for Eve
test_labels = np.hstack((test_alice_label, test_eve_labels))  # (3200,)

products - [ 1044  5152  8120 23236 28894  4615   649   624  8595 16074  1376  4745
  1551    96   112   928  3330    48 11926 20900  8806  5244 26928  2812
  7896   963 23716  5117 29452  4164 19892 23822 16296   106  5040  1581
  2747  1989   392  5170  2160    78  9464  2106  5504   648  2914  2324
  1920  4762 26266  1106 12653 32080 26331 27698 18944 26237  6489 21126
 17134 27002  3617 19804 30656  8052 17817 32308  7573 16496 29264  7120
 12040 14050 28952  3192 12428  7837 23242 27584 15142   528 13863 29642
 18391  3392  2195 11322  2112 14142 29122 24758 29810  7715 30496 11208
 32300  2792 16448 17509 30007  2833 32653  1496 27586  6020  6320 27560
 14424 14241  3995 24050  9438 12196 11224 14476 13698 23174 27136 16632
 26966 13133   960  8464  8160 24013 18616  2240 10658  5184  4516 22288
 31484 26730 30573 12850 26894  1081 27552   990  7380  2574   690   240
  7686 30142 10044 14014 14476  6148 13816 17788 28088 12280 26057  8910
 25232  4806 14820  6588  6837  1344  60

In [9]:
# ----------------- Classification -----------------
# Train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_cirs_flattened, train_labels)

# Predict on test data
predictions = knn.predict(test_cirs_flattened)


In [10]:
# # Initialize and train the SVM classifier
# classifier = SVC(kernel='linear', random_state=42)
# classifier.fit(train_cirs_flattened, train_labels)

# # Predict on test data
# predictions = classifier.predict(test_cirs_flattened)

In [11]:
# ----------------- Evaluation -----------------
# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Missed Detection Rate (MDR)
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0

# False Alarm Rate (FAR)
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0

# Gamma calculation
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0

# Authentication Rate (AR)
denominator = (tp + fn) + gamma * (tn + fp)
AR = (tp + gamma * tn) / denominator if denominator > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")

Classification Accuracy: 46.25%
tp: 20
tn: 17
fp: 23
fn: 20
MDR: 0.575
FAR: 0.5
AR: 0.4625
